# Data node and looping over return values

This example notebook demonstrates the following:

- Use a data node to set simple values (e.g., bool flags)
- Looping over a repeated field with a loop node (similar to a for each loop)

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [flowstate.intrinsic.ai](https://flowstate.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `pick_and_place:pick_and_place_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `pick_and_place:pick_and_place_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

First, connect to your solution and define convenience shortcuts:

In [ ]:
from intrinsic.solutions import behavior_tree as bt
from intrinsic.solutions import deployments
from intrinsic.solutions import cel
from google.protobuf import wrappers_pb2

solution = deployments.connect_to_selected_solution()

executive = solution.executive
skills = solution.skills
world = solution.world

enable_gripper = skills.ai.intrinsic.enable_gripper
move_robot = skills.ai.intrinsic.move_robot
echo_world_nodes = skills.ai.intrinsic.echo_world_nodes

## Create Test Skills

In [ ]:
move_home = bt.Task(name="Move Home", action=move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.home,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ],
    arm_part=world.robot,
))
move_left = bt.Task(name="Move Left", action=move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_left,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ],
    arm_part=world.robot,
))
move_right = bt.Task(name="Move Right", action=move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.view_pose_right,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ],
    arm_part=world.robot,
))

## Data Node with simple values

The data node allows to put simple values (e.g., bool or int) directly on the blackboard.

The common wrapper protos are supported: https://github.com/protocolbuffers/protobuf/blob/main/src/google/protobuf/wrappers.proto

In this example we look at creating a bool flag in one part of a possibly larger process ("Prepare") that is later used in "Main" used to decide how the process proceeds.<br/>
The flag is a `BoolValue` proto that has a `value` field. It is stored on the blackboard under the key name `foo`.

In [ ]:
set_foo_on = bt.Data(name="Set Foo Flag: On",
                  operation=bt.Data.OperationType.CREATE_OR_UPDATE,
                  blackboard_key="foo",
                  proto=wrappers_pb2.BoolValue(value=True))
set_foo_off = bt.Data(name="Set Foo Flag: Off",
                  operation=bt.Data.OperationType.CREATE_OR_UPDATE,
                  blackboard_key="foo",
                  proto=wrappers_pb2.BoolValue(value=False))
#prepare_process = bt.Sequence(name="Prepare", children=[set_foo_off])
prepare_process = bt.Sequence(name="Prepare", children=[set_foo_on])

In the "Main" part of the process a Branch node now decides based on the value of "foo" if the robot should move left or right.

Note that the cel_expression does not point to the `value` field in the `BoolValue` proto. CEL has special handling for wrapper protos and automatically unpacks these. Thus just passing `foo` here is correct and will lead to a Boolean expression representing `foo.value`.

In [ ]:
branch = bt.Branch(if_condition=bt.Blackboard(cel_expression="foo"),
                   then_child=move_left,
                   else_child=move_right)
main_node = bt.Sequence(children=[prepare_process,
                                  bt.Sequence(name="Home", children=[move_home]),
                                  bt.Sequence(name="Main", children=[branch])])
main_node.show()
executive.run(main_node)

Try experimenting with setting the `foo` key on or off in "Prepare" to change the behavior later in the process.

## Looping over lists in return values

Skill return values can contain lists of values (e.g., detected objects). These are represented as repeated fields in a proto.

Looping over return values is achieved by a while loop that stops when the length of a repeated field is reached.

As a first step the `echo_world_nodes` skills allows us to find all frames that start with `target...`. The skill returns the found frames in  a repeated field called `nodes`. We store the result under the name `target_frames`.
This could also be the result of an estimate_pose skill or similar.

In [ ]:
echo_world_node_protos = echo_world_nodes.intrinsic_proto.manipulation.skills
echo_targets = bt.Task(name="Echo Targets", action=echo_world_nodes(object_subset=echo_world_node_protos.ObjectFrameSubset(
    parent_object=world.root,
    frames_of_interest_regex="target.*"),
    return_value_key="target_frames"))

As the next step a generic `move_robot` skill is created by `move_gripper_to_frame`. This moves the gripper to a position roughly 10cm above the target frame.

Note that frame here is a parameter. You can pass in an actual frame or a CelExpression that evaluates to a frame at runtime.

In [ ]:
move_robot_protos = move_robot.intrinsic_proto.skills
motion_planning_protos = move_robot.intrinsic_proto.motion_planning.v1
def move_gripper_to_frame(frame):
    return move_robot(motion_segments=[move_robot_protos.MotionSegment(
        position_equality=motion_planning_protos.PositionEquality(
            moving_frame=world.picobot_gripper.tool_frame,
            target_frame=frame,
            moving_frame_offset=move_robot.intrinsic_proto.Vector3(x=0.0,y=0.0,z=0.1),
            ))])

Here a loop is set up that loops over the returned `nodes` field of the `echo_world_nodes` skill.

We set the loop counter to `loop_counter` and use that in the while condition to loop as long as the `loop_counter` is less than the size of the found target frames in `target_frame.nodes`.
Note that you can freely combine this condition with additional constraints.

Finally when creating the `move_robot` skill we pass in an expression that references the entry at the `loop_counter` in the `target_frames.nodes[]` field.

In [ ]:
loop_over_target_frames = bt.Loop(loop_counter_key="loop_counter",
                                  while_condition=bt.Blackboard(cel_expression="loop_counter < size(target_frames.nodes)"),
                                  do_child=bt.Sequence(children=[
                                      move_home,
                                      bt.Task(name="Move To Target",
                                              action=move_gripper_to_frame(
                                                  cel.CelExpression(expression="target_frames.nodes[loop_counter]")))
                                      ]))

Our final process first finds the target objects and then loops over these.

In [ ]:
main_sequence = bt.Sequence(children=[echo_targets, loop_over_target_frames, move_home])
executive.run(main_sequence)